In [1]:
!pip install hugchat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 860.2 kB/s eta 0:00:00a 0:00:01


In [8]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import openai

openai.api_key = 'test'

openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

AuthenticationError: <empty message>

In [1]:
from hugchat import hugchat
import json
import re
chatbot = hugchat.ChatBot(cookie_path="cookies.json")  # or cookies=[...]
# print(chatbot.chat("HI"))

# # Create a new conversation
# id = chatbot.new_conversation()
# chatbot.change_conversation(id)

# # Get conversation list
# conversation_list = chatbot.get_conversation_list()

In [2]:
with open('vicuna_experiments/annotation_dataset.json', 'r') as f:
    annotation_data = json.loads(f.read())

In [45]:
with open('vicuna_experiments/annotated_data.jsonl', 'r') as f:
    annotated_data = f.readlines()
    annotated_data = [json.loads(data) for data in annotated_data]
    annotated_data.insert(83, {'id': 83, 'Answer': None})

In [11]:
answers = []

In [12]:
answers

[[], []]

In [17]:
from tqdm import tqdm

answer_extractor = lambda x: [re.findall(r'Answer [AB]', answer) for answer in x]
most_common = lambda x: max(set(x), key=x.count)

PROMPT_QUESTION = '''
Please choose the most relevant answer.
The most relevant answer should follow the next rules:
1. The answer should not contradict with context, facts and dialog.
2. The answer should be natural.
3. The answer should not be absurd or meaningless.
Just write Answer A or Answer B.
'''
for sample in tqdm(annotation_data[len(answers):]):
    prompt = '\n'.join([sample['dialog'], 'Answer A:', sample['Answer A'], 
                        'Answer B:', sample['Answer B'], '\n' + PROMPT_QUESTION])
    
    sample_answers = []
    for _ in range(5):
        try:
            sample_answers.append(chatbot.chat(prompt))
        except KeyboardInterrupt:
            raise KeyboardInterrupt
        except:
            pass
    sample_answers = answer_extractor(sample_answers)
    sample_answers = [answer[0] for answer in sample_answers if answer]
    answers.append(most_common(sample_answers))

100%|██████████| 26/26 [05:05<00:00, 11.76s/it]


In [18]:
session_len = [len(data['dialog'][re.match(r'.*Dialog:\n', data['dialog'], re.S).span()[1]:-2].split('\n'))
               for data in annotation_data]

In [25]:
import pandas as pd

bins = [0, 4, 8, 11]

report = pd.DataFrame({'chatbot_answers': answers, 
                       'human_answers': [data['Answer'] for data in annotated_data],
                       'session_len': session_len})
report['bin'] = pd.cut(report['session_len'], bins)
report['chatbot_context'] = report['chatbot_answers'] == [data['Contexted Answer'] for data in annotation_data]
report['human_context'] = report['human_answers'] == [data['Contexted Answer'] for data in annotation_data]
report = report[report['human_answers'].isin(['Answer A',  'Answer B'])]

In [28]:
report.to_csv('huggingchat_answers.csv', index=False)

In [27]:
report.groupby(['bin']).agg({'chatbot_context': 'sum', 'human_context': 'sum'}) \
        .div(report.groupby(['bin']).agg({'chatbot_context': 'count', 'human_context': 'count'})).round(2)

,chatbot_context,human_context
bin,,
"(0, 4]",0.58,0.60
"(4, 8]",0.49,0.63
"(8, 11]",0.45,0.52


In [191]:
(report['chatbot_answers'] == report['human_answers']).sum() / report.shape[0]

0.5641025641025641

In [45]:
sum([annotation_data[i]['Contexted Answer'] == answer for i, answer in enumerate(answers)]) / len(answers)

0.54

In [109]:
annotation_data[0]['dialog'][re.match(r'.*Dialog:\n', annotation_data[0]['dialog'], re.S).span()[1]:]

"Human: How are you and your cats doing today over in Alaska?\nAssistant: Good thanks. How are you and your dad in Virginia? \nHuman: We are doing pretty good thank you for asking. I hope you don't have any headaches today!\n"

In [55]:
annotation_data[0]['dialog']

"Context:\n1. The Assistant has vision problems and has called 911 in the past due to severe headaches.\n2. The Assistant used to have six cats and is now married to a friend from high school.\n3. The Human is from Virginia and is not currently married.\n\n Facts about Human:\nHuman is not married and lives in Virginia.\n\nFacts about Assistant:\nWorks from home\nHas trouble seeing\nHas 200 feet vision\nGets bad headaches\nHas called 911 before\nUsed to have 6 cats\nLives in Alaska\nAssistant has vision problems and gets headaches.\nAssistant is from Alaska and got married to a friend from high school.\n\n Dialog:\nHuman: How are you and your cats doing today over in Alaska?\nAssistant: Good thanks. How are you and your dad in Virginia? \nHuman: We are doing pretty good thank you for asking. I hope you don't have any headaches today!\n"

In [47]:
sum([annotation_data[i]['Contexted Answer'] == answer[0] for i, answer in extracted_answers]) / len(extracted_answers)

0.4765625

In [39]:
import re

extracted_answers = [(i, re.findall(r'Answer [AB]', answer)) for i, answer in enumerate(answers)]
extracted_answers = [answer for answer in extracted_answers if answer[1]]

In [40]:
sum([annotation_data[i]['Contexted Answer'] == answer[0] for i, answer in extracted_answers]) / len(extracted_answers)

0.4765625

In [23]:
answer = chatbot.chat(prompt)

In [24]:
answer

'Answer B.'

In [25]:
sample

{'dialog': "Context:\n1. The Assistant has vision problems and has called 911 in the past due to severe headaches.\n2. The Assistant used to have six cats and is now married to a friend from high school.\n3. The Human is from Virginia and is not currently married.\n\n Facts about Human:\nHuman is not married and lives in Virginia.\n\nFacts about Assistant:\nWorks from home\nHas trouble seeing\nHas 200 feet vision\nGets bad headaches\nHas called 911 before\nUsed to have 6 cats\nLives in Alaska\nAssistant has vision problems and gets headaches.\nAssistant is from Alaska and got married to a friend from high school.\n\n Dialog:\nHuman: How are you and your cats doing today over in Alaska?\nAssistant: Good thanks. How are you and your dad in Virginia? \nHuman: We are doing pretty good thank you for asking. I hope you don't have any headaches today!\n",
 'Answer A': 'Assistant: I am having a headache right now. My vision is messed up and I can hardly see. ',
 'Answer B': "Assistant: Thank y

In [29]:
with open('answers.json', 'r') as f:
    answers = json.loads(f.read())

In [31]:
from tqdm import tqdm

answer_extractor = lambda x: [re.findall(r'Answer [AB]', answer) for answer in x]
most_common = lambda x: max(set(x), key=x.count)

extracted_answers = []
for sample_answers in tqdm(answers):
    sample_answers = answer_extractor(sample_answers)
    sample_answers = [answer[0] for answer in sample_answers if answer]
    extracted_answers.append(most_common(sample_answers))

100%|██████████| 150/150 [00:00<00:00, 112851.23it/s]


In [37]:
sum([annotation_data[i]['Contexted Answer'] == answer for i, answer in enumerate(extracted_answers)]) / len(extracted_answers)

0.5266666666666666

In [54]:
correct_annotated_data = [(i, data) for i, data in enumerate(annotated_data) if data['Answer'] in ['Answer A', 'Answer B']]

In [56]:
import pandas as pd

dataset = pd.read_csv('huggingchat_answers.csv')

In [57]:
dataset = dataset.rename({'chatbot_answers': 'open_assistant_answers', 'chatbot_context': 'open_assistant_context'}, axis=1)

In [58]:
dataset['gpt_4_answers'] = [extracted_answers[i] for i, answer in correct_annotated_data]

In [63]:
dataset['open_assistant_context'] = dataset['open_assistant_answers'] == [data['Contexted Answer'] for i, data in correct_annotated_data]
dataset['human_context'] = dataset['human_answers'] == [data['Contexted Answer'] for i, data in correct_annotated_data]
dataset['gpt_4_context'] = dataset['gpt_4_answers'] == [data['Contexted Answer'] for i, data in correct_annotated_data]

In [65]:
dataset.to_csv('dialog_evaluation.csv', index=False)

In [66]:
len(correct_annotated_data)

117